In [ ]:
!pip install langchain pypdf unstructured pdf2image docx2txt pdfminer
!pip install tiktoken
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 30.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.7/228.7 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.9/457.9 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 22.8 M

In [ ]:
import gensim
import nltk
import pickle
from gensim import corpora
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')


from pypdf import PdfReader
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.llms import OpenAI

import re
nltk.download('all')

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

In [ ]:
import pandas as pd
pos_file = 'data/reviews_llm/negative_reviews_llm.csv'
neg_file = 'data/reviews_llm/positive_reviews_llm.csv'
df = pd.read_csv(pos_file)
df.columns

Index(['Unnamed: 0', 'review'], dtype='object')

In [ ]:
pd.read_csv(pos_file)

,Unnamed: 0,review
0,0,is a good idea to get off from our station.
1,1,I want to improve.1
2,2,seems to have increased the number of users i...
3,3,I think the sound of the name Daimon is also ...
4,4,I was surprised.
...,...,...
1381,1381,It is good because the antenna shop changes!C...
1382,1382,"It is now a soup stock Tokyo, but the curry w..."
1383,1383,picture is a cafe on the third floor.The staf...
1384,1384,Station premises are relatively clean. The go...


In [ ]:
def preprocessing(a):
  # get rid of non english character
  only_en_pos_review = re.sub('[^a-zA-z]', ' ', a)
  # change it to lower case
  no_cap_pos_review = only_en_pos_review.lower()
  # tokenizing
  word_tokens = nltk.word_tokenize(no_cap_pos_review)
  # pos tagging
  tokens_pos = nltk.pos_tag(word_tokens)
  # only get NN, VB, JJ (noun, verb, adjective)
  final_words = []
  for word, pos in tokens_pos:
    if  pos == 'NN':
      final_words.append(word)
    elif pos == 'VB':
      final_words.append(word)
    elif pos == 'JJ':
      final_words.append(word)
  # lemmatize
  lemma = nltk.WordNetLemmatizer()
  lemmatized_words = []
  for word in final_words:
    new_word = lemma.lemmatize(word)
    lemmatized_words.append(new_word)
  # stop words
  stopwords_list = stopwords.words(['english', 'spanish'])
  # adding stop words
  add_stopwords_list = ['station', 'seoul', 'yokohama','train', 'line', 'place', 'tokyo', 'newyork',
                        'york', 'time', 'kyoto', 'hankyu', 'osaka', 'good', 'shibuya',
                        'lot', 'many']
  stopwords_list = stopwords_list +  add_stopwords_list

  unique_words = set(lemmatized_words)
  final_words = lemmatized_words

  # get rid of stop words
  for word in unique_words:
      if word in stopwords_list or len(word) < 3: # delete when it is stopwords or words with two letters
          while word in final_words: final_words.remove(word)

  return final_words

In [ ]:
def get_topic_lists_from_csv(file, num_topics, words_per_topic):
    """
    Extracts topics and their associated words from a PDF document using the
    Latent Dirichlet Allocation (LDA) algorithm.

    Parameters:
        file (str): The path to the PDF file for topic extraction.
        num_topics (int): The number of topics to discover.
        words_per_topic (int): The number of words to include per topic.

    Returns:
        list: A list of num_topics sublists, each containing relevant words
        for a topic.
    """
    # Load the pdf file
    '''
    loader = CSVLoader(file_path= file, csv_args={
      'delimiter': ',',
      'quotechar': '"',
      'fieldnames': ['Unnamed: 0', 'statnNm', 'rating', 'review', 'review_count']
    })
    '''

    # Extract the text from each page into a list. Each page is considered a document
    data = pd.read_csv(file)
    documents= []
    for i in data['review']:
      documents.append(i)

    # Preprocess the documents
    processed_documents = [preprocessing(sentence) for sentence in documents]
    #processed_documents = [preprocess(doc, stop_words) for doc in documents]
    # Create a dictionary and a corpus
    dictionary = Dictionary(processed_documents)
    dictionary.filter_extremes(keep_n = 2000, no_below=5, no_above=0.5)
    corpus = [dictionary.doc2bow(doc) for doc in processed_documents]

    # Build the LDA model
    lda_model = LdaModel(
        corpus,
        num_topics=num_topics,
        id2word=dictionary,
        passes=15,
        random_state = 7
        )
    
    # saving model and change the name for the model
    # lda_model.save("lda_model")

    # Retrieve the topics and their corresponding words
    topics = lda_model.print_topics(num_words=words_per_topic)

    # Store each list of words from each topic into a list
    topics_ls = []
    for topic in topics:
        print(topic)
        words = topic[1].split("+")
        topic_words = [word.split("*")[1].replace('"', '').strip() for word in words]
        topics_ls.append(topic_words)

    return topics_ls

In [ ]:
pos_template = '''Describe the postive topic of each of the {num_topics}
        double-quote delimited lists in a simple phrase and also write down
        ten possible words that can describe the topic. The topics must not
        include city name or any pronouns. The lists are the result of an
        algorithm for topic discovery.

        Do not provide an introduction or a conclusion, only describe the
        topics. Do not mention the word "topic" when describing the topics.
        Use the following template for the response.

        1: <<<(sentence describing the topic)>>>
        - <<<(whether topic is positive or negative)>>>
        - <<<(words describing the topic)>>>

        2: <<<(sentence describing the topic)>>>
        - <<<(whether topic is positive or negative)>>>
        - <<<(words describing the topic)>>>


        ...

        n: <<<(sentence describing the topic)>>>
        - <<<(whether topic is positive or negative)>>>
        - <<<(words describing the topic)>>>


        Lists: """{string_lda}""" '''

neg_template = '''Describe the negative topics of each of the {num_topics}
        double-quote delimited lists in a simple phrase and also write down
        ten possible words that can describe the topic.The topics must not
        include city name or any pronouns. The lists are the result of an
        algorithm for topic discovery.

        Do not provide an introduction or a conclusion, only describe the
        topics. Do not mention the word "topic" when describing the topics.
        Use the following template for the response.

        1: <<<(sentence describing the topic)>>>
        - <<<(whether topic is positive or negative)>>>
        - <<<(words describing the topic)>>>

        2: <<<(sentence describing the topic)>>>
        - <<<(whether topic is positive or negative)>>>
        - <<<(words describing the topic)>>>


        ...

        n: <<<(sentence describing the topic)>>>
        - <<<(whether topic is positive or negative)>>>
        - <<<(words describing the topic)>>>


        Lists: """{string_lda}""" '''

In [ ]:
# 파라미터도 똑같이 맞추었는데 lda_tm_실험실 파일에서 나오는 토픽 단어 분포랑 이 파일에서 나오는 거랑 다르게 나옴.
# 일시적인 해결책 => lda_tm_실험실 파일에서 topics_ls 부분을 복사해서 아래 함수 list_of_topicwords에 붙여넣기 통해서 진행
# 조금 번거로워서 코드 어떻게 해서 맞춰보겠음

In [ ]:
def topics_from_csv(llm, template, file, num_topics, words_per_topic):
    """
    Generates descriptive prompts for LLM based on topic words extracted from a
    PDF document.

    This function takes the output of `get_topic_lists_from_pdf` function,
    which consists of a list of topic-related words for each topic, and
    generates an output string in table of content format.

    Parameters:
        llm (LLM): An instance of the Large Language Model (LLM) for generating
        responses.
        file (str): The path to the PDF file for extracting topic-related words.
        num_topics (int): The number of topics to consider.
        words_per_topic (int): The number of words per topic to include.

    Returns:
        str: A response generated by the language model based on the provided
        topic words.
    """

    # Extract topics and convert to string
    list_of_topicwords = get_topic_lists_from_csv(file,num_topics,words_per_topic)
    string_lda = ""
    for list in list_of_topicwords:
        string_lda += str(list) + "\n"

    # Create the template
    template_string = template

    # LLM call
    
    prompt_template = ChatPromptTemplate.from_template(template_string)
    chain = LLMChain(llm=llm, prompt=prompt_template)
    response = chain.run({
        "string_lda" : string_lda,
        "num_topics" : num_topics
        })

    return response

In [ ]:
openai_key = '' # use personal key
llm = OpenAI(openai_api_key=openai_key, max_tokens=-1)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [ ]:
# change number of topics

num_topics = 4
# num_topics = 5
# num_topics = 6
words_per_topic = 30

summary = topics_from_csv(llm, pos_template, pos_file, num_topics, words_per_topic)

['station', 'good', 'line', 'airport', 'convenient', 'subway', 'many', 'train', 'lot', 'place']
['station', 'good', 'new', 'train', 'place', 'subway', 'clean', 'beautiful', 'many', 'city']
['station', 'good', 'many', 'lot', 'delicious', 'enjoy', 'day', 'high', 'shop', 'walk']
['station', 'line', 'good', 'convenient', 'clean', 'many', 'place', 'easy', 'transfer', 'use']



In [ ]:
summary

'\n1: <<<(A convenient and well-connected transportation hub)>>>\n- <<<(positive)>>>\n- <<<(station, good, line, airport, convenient, subway, many, train, lot, place)>>>\n\n2: <<<(A modern and efficient train station)>>>\n- <<<(positive)>>>\n- <<<(station, good, new, train, place, subway, clean, beautiful, many, city)>>>\n\n3: <<<(A bustling and lively food market)>>>\n- <<<(positive)>>>\n- <<<(station, good, many, lot, delicious, enjoy, day, high, shop, walk)>>>\n\n4: <<<(An easy and efficient subway system)>>>\n- <<<(positive)>>>\n- <<<(station, line, good, convenient, clean, many, place, easy, transfer, use)>>>\n'

In [ ]:
# num_topics = 4
# num_topics = 5
num_topics = 6
 
words_per_topic = 30

summary = topics_from_csv(llm, neg_template, neg_file, num_topics, words_per_topic)

['station', 'ticket', 'place', 'good', 'difficult', 'gate', 'elevator', 'lot', 'home', 'old']
['station', 'train', 'lot', 'subway', 'good', 'many', 'dirty', 'street', 'new', 'night']
['station', 'line', 'transfer', 'many', 'train', 'bad', 'good', 'subway', 'use', 'exit']
['bad', 'high', 'infection', 'narrow', 'station', 'shop', 'store', 'risk', 'business', 'area']



In [ ]:
summary

'\n\n1: <<<The station is a difficult and crowded place that is often confusing and overwhelming.>>>\n- Negative\n- Difficult, crowded, confusing, overwhelming, busy, hectic, chaotic, stressful, frustrating, overwhelming\n\n2: <<<Taking the train can be a dirty and unpleasant experience, especially at night.>>>\n- Negative\n- Dirty, unpleasant, crowded, busy, chaotic, stressful, frustrating, overwhelming, uncomfortable, unclean\n\n3: <<<Navigating the station can be a challenge with its many lines and transfers, making it difficult to use.>>>\n- Negative\n- Challenging, confusing, difficult, overwhelming, crowded, chaotic, stressful, frustrating, complicated, overwhelming\n\n4: <<<The station area has a high risk of infection due to its narrow and crowded streets, making it a bad place to do business.>>>\n- Negative\n- High risk, infection, narrow, crowded, dirty, unclean, unhealthy, unsanitary, hazardous, polluted'